# Bootstrapping a sphere 4-pt function

This notebook displays the output of running the file [bootstrap.jl](./bootstrap.jl).
We show the source code here for convenience:


```julia
using BootstrapVirasoro

# set the precision of high-precision floats
setprecision(BigFloat, 40, base=10)

β = big"0.3" + big"0.6"*im
# define a central charge
c = CC(β = β)

V12 = Field(c, r=1, s=2, diagonal=true) # the external, degenerate field
P = big"0.67"
VP = Field(c, P=P) # the external diagonal field
VPpm = [Field(c, P = P + pm*1/(2β)) for pm in (-1, 1)] # shifted fields
Vd = [Field(c, r=1, s=1, diagonal=true),
      Field(c, r=1, s=3, diagonal=true),
      Field(c, r=1, s=5, diagonal=true)] # degenerate fields
# list of all fields that can go in channels
# vcat concatenates arrays.
V = vcat(VPpm, Vd) 

Cor   = Correlation(V12, VP, V12, VP, 70) # define the correlation

# We need to define ChannelSpectrum objects for each channel.
# They are created as ChannelSpectrum(correlation, channel, listoffields, function)
# where the function is a function that takes as input a field and outputs a block object.
fs(V) = NCBlock(Cor, :s, V) # map fields to blocks
Cs    = ChannelSpectrum(Cor, :s, V, fs) # create s-channel spectrum
ft(V) = NCBlock(Cor, :t, V)
Ct    = ChannelSpectrum(Cor, :t, V, ft) # t-channel
fu(V) = NCBlock(Cor, :u, V)
Cu    = ChannelSpectrum(Cor, :u, V, fu) # u-channel

sol = solve_bootstrap(Channels(Cs, Ct, Cu)) # setup and solve the crossing equations
sol.str_cst # return the structure constants
```

We can run the code which is inside of this source file with `include`, and the return value (last line) of this file will automatically be printed in the output of the cell:

In [1]:
include("bootstrap.jl")

 Channel s
┌───────────────────┬────────────────────┬───────────┐
│ Field             │ Structure constant │ Rel. err. │
├───────────────────┼────────────────────┼───────────┤
│ <1, 5>            │ 0                  │ 3.3       │
│ <1, 3>            │ 0                  │ 3.3       │
│ <1, 1>            │ 0                  │ 8.1       │
│ P = (0.34+0.67im) │ -73.6374+41.7927im │ 1.2e-21   │
│ P = (1-0.67im)    │ 1.0+0.0im          │ 0         │
└───────────────────┴────────────────────┴───────────┘
 Channel t
┌───────────────────┬────────────────────┬───────────┐
│ Field             │ Structure constant │ Rel. err. │
├───────────────────┼────────────────────┼───────────┤
│ <1, 5>            │ 0                  │ 3         │
│ <1, 3>            │ 0                  │ 3.6       │
│ <1, 1>            │ 0                  │ 6.1       │
│ P = (0.34+0.67im) │ -73.6374+41.7927im │ 1.2e-21   │
│ P = (1-0.67im)    │ 1.0+3.1099e-24im   │ 1.1e-21   │
└───────────────────┴────────────────────┴─

We correctly find that the degenerate field $\langle 1, 5 \rangle$ has a vanishing structure constant, and that the structure constants in the three channels are equal.
Alternatively we could have imposed in the first place that the structure constants in all channels by passing `rels=:st` to the `solve_bootstrap` call (`:` is the Julia syntax for declaring a `Symbol`):

```julia
solve_bootstrap(Channels(Cs, Ct, Cu), rels=:stu)
```

Notice also that the code automatically decided to normalise one of the structure constants to 1. We could choose a different normalisation or fixed more known structure constants by using the `fix` argument:

```julia
solve_bootstrap(Channels(Cs, Ct, Cu), rels=:stu, fix=[(:s, V12, 1.32)])
```

If we wanted to set several structure constants we would pass a Vector of triples. For instance,

```julia
fix = [(:s, V12, 1.32), (:t, VP, 0.8)]
```

would fix the s-channel structure constant of the $V_{\langle 1, 2 \rangle}$ field to 1.32 and the t-channel structure constant of $V_P$ to 0.8.

In [ ]:
sol = solve_bootstrap(Channels(Cs, Ct, Cu), rels=:st)
sol.str_cst

CompositeException: TaskFailedException

    nested task error: UndefRefError: access to undefined reference
    Stacktrace:
     [1] getindex
       @ ./essentials.jl:920 [inlined]
     [2] iterate (repeats 2 times)
       @ ./array.jl:901 [inlined]
     [3] iterate
       @ ./generator.jl:45 [inlined]
     [4] collect(itr::Base.Generator{Vector{LeftRight{BootstrapVirasoro.PosCache}}, BootstrapVirasoro.var"#evaluate_blocks##6#evaluate_blocks##7"{BootstrapVirasoro.FactorizedBlock{Complex{BigFloat}}}})
       @ Base ./array.jl:790
     [5] macro expansion
       @ ~/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/4_bootstrapsystem.jl:367 [inlined]
     [6] (::BootstrapVirasoro.var"#evaluate_blocks##2#evaluate_blocks##3"{BootstrapVirasoro.var"#evaluate_blocks##4#evaluate_blocks##5"{Vector{LeftRight{BootstrapVirasoro.PosCache}}, Vector{Block{Complex{BigFloat}}}, Vector{Dict{Union{ConformalDimension, Field}, Vector{Complex{BigFloat}}}}, UnitRange{Int64}}})(tid::Int64; onethread